# Team win history GRU

Choose two teams, the history will first be displayed as (for example) 0,1,0,..,1,0 etc.
Then these are compiled and a GRU is made to predict the probability of future win of that team against a challenger...

In [4]:
# Python imports
import pandas as pd
import numpy as np

In [5]:
dat = pd.read_csv('matchdata.csv',encoding = "ISO-8859-1")


/home/max/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (82,83,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# Need a function that pulls out only a certain teams data:

def team_data(team): # probably by index...
    df = dat.loc[(dat.ix[:,3] == team) |  (dat.ix[:,4] == team) ]
    return(df)


In [7]:
team_data('fnatic')

,Day,Match ID,Map,Team1,Team2,Winning Team,Score Team 1 as T,Score Team 2 as CT,Score Team 1 as CT,Score Team 2 as T,...,Player 12 Kills,Player 12 Deaths,Player 12 KDA,Player 12 ADR,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87
8,16953,TODO,mirage,fnatic,SK,fnatic,10,5,6,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,16952,TODO,train,SK,fnatic,SK,9,6,7,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,16916,TODO,overpass,HellRaisers,fnatic,fnatic,4,11,6,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,16915,TODO,cache,fnatic,HellRaisers,HellRaisers,7,8,6,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213,16748,TODO,cobblestone,EnVyUs,fnatic,EnVyUs,11,4,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
219,16742,TODO,dust2,fnatic,EnVyUs,EnVyUs,1,14,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
283,16678,TODO,cobblestone,Virtus.pro,fnatic,fnatic,6,9,1,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287,16674,TODO,cache,fnatic,Virtus.pro,Virtus.pro,7,8,5,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,16603,TODO,cache,PENTA,fnatic,fnatic,3,12,9,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
368,16593,TODO,overpass,fnatic,PENTA,fnatic,7,8,9,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
def win_series(team1,team2):
    score_diff = []
    
    
    # find only games where team 1 and team 2 are playing
    df = dat.loc[((dat.ix[:,3] == team1) |  (dat.ix[:,4] == team1) ) & ((dat.ix[:,3] == team2) |  (dat.ix[:,4] == team2) )]
    df = df.reset_index(drop = True) 
    # now for each row find if they won or lost: winning team == team1
    for i in range(len(df)):
        S1_score = np.sum(df.ix[i,(6,8)]) # for team on side1
        S2_score = np.sum(df.ix[i,(7,9)]) # for team on side 2
        
        # calculate which side team1 is on
        if team1 == df.ix[i,3]: # then t1 on side 1
            score_diff.append(S1_score - S2_score)
        else: # t1 on side 2
            score_diff.append(S2_score - S1_score)
            
        
    # now reverse the win vector to be oldest to newest
    diff = np.flipud(score_diff)
    ins= diff[:len(diff)-1]
    targs = diff[1:len(diff)] 
    
    return ins, targs
    

In [9]:
''' sequence padding code '''
from __future__ import absolute_import
# -*- coding: utf-8 -*-
import random
from six.moves import range


def pad_sequences(sequences, maxlen=None, dtype='int32',
                  padding='pre', truncating='pre', value=0.):
    '''Pads each sequence to the same length:
    the length of the longest sequence.

    If maxlen is provided, any sequence longer
    than maxlen is truncated to maxlen.
    Truncation happens off either the beginning (default) or
    the end of the sequence.

    Supports post-padding and pre-padding (default).

    # Arguments
        sequences: list of lists where each element is a sequence
        maxlen: int, maximum length
        dtype: type to cast the resulting sequence.
        padding: 'pre' or 'post', pad either before or after each sequence.
        truncating: 'pre' or 'post', remove values from sequences larger than
            maxlen either in the beginning or in the end of the sequence
        value: float, value to pad the sequences to the desired value.

    # Returns
        x: numpy array with dimensions (number_of_sequences, maxlen)
    '''
    lengths = [len(s) for s in sequences]

    nb_samples = len(sequences)
    if maxlen is None:
        maxlen = np.max(lengths)

    # take the sample shape from the first non empty sequence
    # checking for consistency in the main loop below.
    sample_shape = tuple()
    for s in sequences:
        if len(s) > 0:
            sample_shape = np.asarray(s).shape[1:]
            break

    x = (np.ones((nb_samples, maxlen) + sample_shape) * value).astype(dtype)
    for idx, s in enumerate(sequences):
        if len(s) == 0:
            continue  # empty list was found
        if truncating == 'pre':
            trunc = s[-maxlen:]
        elif truncating == 'post':
            trunc = s[:maxlen]
        else:
            raise ValueError('Truncating type "%s" not understood' % truncating)

        # check `trunc` has expected shape
        trunc = np.asarray(trunc, dtype=dtype)
        if trunc.shape[1:] != sample_shape:
            raise ValueError('Shape of sample %s of sequence at position %s is different from expected shape %s' %
                             (trunc.shape[1:], idx, sample_shape))

        if padding == 'post':
            x[idx, :len(trunc)] = trunc
        elif padding == 'pre':
            x[idx, -len(trunc):] = trunc
        else:
            raise ValueError('Padding type "%s" not understood' % padding)
    return x


In [130]:
X, T= win_series('EnVyUs','fnatic')
X1, T1 = win_series('EnVyUs','Natus Vincere')
X2, T2 = win_series('EnVyUs','Luminosity')
X3, T3 = win_series('EnVyUs','Virtus.pro')
X4, T4 = win_series('EnVyUs','TSM')
X5, T5 = win_series('EnVyUs','Astralis')
X6, T6 = win_series('EnVyUs','NiP')
X7, T7 = win_series('fnatic','Luminosity')
X8, T8 = win_series('EnVyUs','SK')
X9, T9 = win_series('Astralis','NiP')
X10, T10 = win_series('Virtus.pro','NiP')
X11, T11 = win_series('Virtus.pro','Luminosity')
X12, T12 = win_series('Virtus.pro','Natus Vincere')
X13, T13 = win_series('Virtus.pro','Astralis')
X14, T14 = win_series('Virtus.pro','TSM')
X15, T15 = win_series('fnatic','SK')
X16, T16 = win_series('fnatic','Natus Vincere')
X17, T17 = win_series('Astralis','SK')
X18, T18 = win_series('Astralis','Luminosity')
X19, T19 = win_series('Natus Vincere','Luminosity')
X20, T20 = win_series('Natus Vincere','NiP')
X21, T21 = win_series('Natus Vincere','SK')



# now zero pad all to be the same shape:



# combine into an array:

ins =  pad_sequences(np.array([X.tolist(),X1.tolist(),X2.tolist(),X3.tolist(),X4.tolist(),X5.tolist(),X6.tolist(),X7.tolist(),X8.tolist(),X9.tolist(),X10.tolist(),X11.tolist(),X12.tolist(),X13.tolist(),X14.tolist(),X15.tolist(),X16.tolist(),X17.tolist(),X18.tolist(),X19.tolist(),X20.tolist(),X21.tolist()]) )
targs = pad_sequences(np.array([T.tolist(),T1.tolist(),T2.tolist(),T3.tolist(),T4.tolist(),T5.tolist(),T6.tolist(),T7.tolist(),T8.tolist(),T9.tolist(),T10.tolist(),T11.tolist(),T12.tolist(),T13.tolist(),T14.tolist(),T15.tolist(),T16.tolist(),T17.tolist(),T18.tolist(),T19.tolist(),T20.tolist(),T21.tolist()]) )



Just normalize it G

In [131]:
from sklearn.preprocessing import MinMaxScaler

In [149]:
insNorm = ins/16
targsNorm = targs/16


In [150]:
insNorm.shape

(22, 59)

In [151]:
''' split into testing and training sets '''
len(ins)
split = 0.7
Ntrain = int(round(len(ins)*split,0))

train_ins = insNorm[:Ntrain]
test_ins = insNorm[Ntrain:]
train_T = targsNorm[:Ntrain]
test_T = targsNorm[Ntrain:]

test_true = targs[Ntrain:]

In [152]:
test_true.shape

(7, 59)

In [140]:
# Run the model of GRU
# Keras imports 
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM,GRU, Embedding

In [153]:
in_neurons = 1
hidden_neurons = 2
 
model = Sequential()  
model.add(GRU(output_dim=hidden_neurons, input_dim= in_neurons, return_sequences=True, consume_less='gpu')) 
model.add(GRU(output_dim=hidden_neurons, input_dim= hidden_neurons, return_sequences=True, consume_less='gpu')) 
model.compile(loss="mse", optimizer= 'rmsprop' ) 

In [158]:
''' train this fella '''
model.fit(np.reshape(train_ins,(len(train_ins),train_ins.shape[1],1) ), np.reshape(train_T,(len(train_T),train_T.shape[1],1) ), batch_size=2, nb_epoch = 100, validation_split=0.1)  


Train on 13 samples, validate on 2 samples
Epoch 1/100
13/13 [==============================] - 0s - loss: 0.0665 - val_loss: 0.0671
Epoch 2/100
13/13 [==============================] - 0s - loss: 0.0665 - val_loss: 0.0671
Epoch 3/100
13/13 [==============================] - 0s - loss: 0.0665 - val_loss: 0.0671
Epoch 4/100
13/13 [==============================] - 0s - loss: 0.0665 - val_loss: 0.0671
Epoch 5/100
13/13 [==============================] - 0s - loss: 0.0665 - val_loss: 0.0671
Epoch 6/100
13/13 [==============================] - 0s - loss: 0.0665 - val_loss: 0.0671
Epoch 7/100
13/13 [==============================] - 0s - loss: 0.0665 - val_loss: 0.0671
Epoch 8/100
13/13 [==============================] - 0s - loss: 0.0665 - val_loss: 0.0671
Epoch 9/100
13/13 [==============================] - 0s - loss: 0.0665 - val_loss: 0.0671
Epoch 10/100
13/13 [==============================] - 0s - loss: 0.0665 - val_loss: 0.0670
Epoch 11/100
13/13 [==============================] - 0s

In [155]:
test_true.shape

(7, 59)

In [156]:
test_true.shape

(7, 59)

In [164]:
team_ix = 6
Tshape = np.shape(train_ins)[1]
predicted = model.predict(np.reshape(test_ins,(np.shape(test_ins)[0],Tshape,1)))

preds = pd.DataFrame()
preds['Real'] = np.reshape(test_true[team_ix],(Tshape) )
preds['Norm'] = np.reshape(test_T[team_ix],(Tshape) )
preds['Predicted'] = np.reshape(predicted[team_ix],(Tshape) )
print(preds)

    Real    Norm  Predicted
0      0  0.0000  -0.000133
1      0  0.0000  -0.000450
2      0  0.0000  -0.000702
3      0  0.0000  -0.000867
4      0  0.0000  -0.000969
5      0  0.0000  -0.001031
6      0  0.0000  -0.001067
7      0  0.0000  -0.001088
8      0  0.0000  -0.001101
9      0  0.0000  -0.001108
10     0  0.0000  -0.001112
11     0  0.0000  -0.001115
12     0  0.0000  -0.001116
13     0  0.0000  -0.001117
14     0  0.0000  -0.001118
15     0  0.0000  -0.001118
16     0  0.0000  -0.001118
17     0  0.0000  -0.001118
18     0  0.0000  -0.001118
19     0  0.0000  -0.001118
20     0  0.0000  -0.001119
21     0  0.0000  -0.001119
22     0  0.0000  -0.001119
23     0  0.0000  -0.001119
24     0  0.0000  -0.001119
25     0  0.0000  -0.001119
26     0  0.0000  -0.001119
27     0  0.0000  -0.001119
28     0  0.0000  -0.001119
29     0  0.0000  -0.001119
30     0  0.0000  -0.001119
31     0  0.0000  -0.001119
32     0  0.0000  -0.001119
33     0  0.0000  -0.001119
34     0  0.0000  -0

In [227]:
INP = win_series('EnVyUs','G2')
predicted = model.predict(np.reshape(np.array(INP) , (2,len(INP[0]),1)) )[0]


In [228]:
predicted

array([[ 0.00726403],
       [ 0.01320932],
       [ 0.54285729],
       [ 0.54242665],
       [ 0.54244822],
       [ 0.54244715],
       [ 0.49628255],
       [ 0.45058441],
       [ 0.40618879],
       [ 0.36383224],
       [ 0.32410753],
       [ 0.5488987 ],
       [ 0.54221153],
       [ 0.54245901],
       [ 0.54244661],
       [ 0.49628198]], dtype=float32)

In [234]:
INP = win_series('EnVyUs','HellRaisers')

predicted = model.predict(np.reshape(np.array(INP) , (2,len(INP[0]),1)) )[0]
TTshape = len(predicted)
preds = pd.DataFrame()
preds['Real'] = np.reshape(INP[0],(TTshape) )
preds['Predicted'] = np.reshape(predicted,(TTshape) )
print(preds)

    Real  Predicted
0      1   0.542201
1      1   0.542459
2      1   0.542447
3      1   0.542447
4      1   0.542447
5      0   0.496283
6      0   0.450584
7      1   0.546245
8      1   0.542259
9      0   0.496094
10     1   0.544562
11     1   0.542341
12     1   0.542453
13     1   0.542447
14     1   0.542447
